In [ ]:
# Import python packages
import pandas as pd
from simple_salesforce import Salesforce,SalesforceMalformedRequest
import json
import datetime
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import (Mail, Attachment, FileContent, FileName, FileType, Disposition, ContentId, To)
import base64

In [ ]:
# Create an active session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Create empty lists
errorlist = []
failed_ids = []

In [ ]:
-- Get sendgrid_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.email_reports;
CREATE OR REPLACE function sendgrid_api_key_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SENDGRID_APIS_ACCESS_INTEGRATION_EMAIL)
SECRETS = ('cred' = sendgrid_dev_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sendgrid_api_key_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.email_reports.sendgrid_api_key_secret()"
sendgrid_api_key = session.sql(api_secret_query).collect()[0][0]

In [ ]:
-- Get sf_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.ARVINDR;
CREATE OR REPLACE function sf_hcorp_api_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SALESFORCE_APIS_ACCESS_INTEGRATION)
SECRETS = ('cred' = PYTHON_SALESFORCE_PRD_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sf_hcorp_api_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.ARVINDR.sf_hcorp_api_secret()"
sf_token = session.sql(api_secret_query).collect()[0][0]

In [ ]:
-- Get sf_password from Snowflake secrets
use schema PYTHON_WORKLOADS.ARVINDR;
CREATE OR REPLACE function sf_hcorp_pass_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SALESFORCE_APIS_ACCESS_INTEGRATION)
SECRETS = ('cred' = PYTHON_SALESFORCE_PRD)
AS
$$
import _snowflake
def get_secret_1():
    username_password_object = _snowflake.get_username_password('cred');
    username_password_dictionary = {}
    username_password_dictionary["Username"] = username_password_object.username
    username_password_dictionary["Password"] = username_password_object.password
    return username_password_dictionary
$$;
grant usage on function sf_hcorp_pass_secret() to role svc_engineer_prd;

In [ ]:
# Store sf_password in variable
api_secret_query = "select PYTHON_WORKLOADS.ARVINDR.sf_hcorp_pass_secret()"
sf_pass = session.sql(api_secret_query).collect()[0][0]
sf_pass = sf_pass.replace("\'", "\"")
sf_creds = json.loads(sf_pass)
sf_password = sf_creds["Password"]
sf_uname = sf_creds["Username"]

In [ ]:
-- Get sf_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.ARVINDR;
CREATE OR REPLACE function sf_hcorp_api_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SALESFORCE_APIS_ACCESS_INTEGRATION)
SECRETS = ('cred' = PYTHON_SALESFORCE_PRD_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sf_hcorp_api_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.ARVINDR.sf_hcorp_api_secret()"
sf_token = session.sql(api_secret_query).collect()[0][0]

In [ ]:
# Connect to Salesforce - PRD
sf = Salesforce (username=sf_uname, password=sf_password, security_token=sf_token) 

In [ ]:
-- Retreive the customer sucess object data
select MID,YM, bucket, propensity_to_quit, customer_lifetime_value,rank_of_priority from bi.ci.TDT_PROPENSITY_TO_QUIT_DATA;

In [ ]:
# Convert and format the retreived data into a dataframe
cs_obj_mid_df = cell8.to_pandas()
current_M=(datetime.datetime.today()- datetime.timedelta(days=30)).replace(day=1).strftime("%b")
cs_obj_mid_df = cs_obj_mid_df.fillna('')
cs_obj_mid_df['MID__c'] = "" + current_M.upper() + cs_obj_mid_df['MID'] + ""
cs_obj_mid_df = cs_obj_mid_df.rename(columns={'YM':'Period_Month_Year__c','BUCKET':'Bucket__c','PROPENSITY_TO_QUIT':'Propensity_to_Quit__c','CUSTOMER_LIFETIME_VALUE':'Customer_Lifetime_Value__c','RANK_OF_PRIORITY':'Rank_of_Priority__c'})
cs_obj_mid_df['Period_Month_Year__c'] = cs_obj_mid_df['Period_Month_Year__c'].astype('str')

In [ ]:
def records_insert(sf,records):
    try:
        results = sf.bulk.customer_success__c.insert(records)
    except SalesforceMalformedRequest as e:
        print("Record insertion failed")
        return [],[]

    successful_records=[]
    failed_records=[]

    for record,result in zip(records,results):
        if result['success']:
            successful_records.append(record)
        else:
            failed_records.append((record,result['errors']))
    
    return successful_records,failed_records

In [ ]:
# Initial Load with MIDs
cs_obj_mid_c_df=cs_obj_mid_df[['Period_Month_Year__c','Bucket__c','Propensity_to_Quit__c','Rank_of_Priority__c','Customer_Lifetime_Value__c','MID__c']]
cs_obj_mid_c_df['Product__r'] = '{"MID__c":"'+ cs_obj_mid_df['MID']+'"}'
cs_obj_mid_c_df['Product__r'] = cs_obj_mid_c_df['Product__r'].apply(json.loads)
cs_obj_mid_c_dict=cs_obj_mid_c_df.to_dict(orient='records')
successful_records_initial,failed_records_initial=records_insert(sf,cs_obj_mid_c_dict)
if successful_records_initial:
    initial_record_count=len(successful_records_initial)
else:
    initial_record_count=0

In [ ]:
print(initial_record_count)

In [ ]:
# Try to insert failed records to CPBM
cs_obj_cpbm=[]
for record,errors in failed_records_initial:
    if 'Product__r' in record:
        record['Product__r']=json.loads('{"UniqueKeyRecTypeMID__c":"012f1000000PVPhAAO' + record['MID__c'][3:]+'"}')
    cs_obj_cpbm.append(record)

if cs_obj_cpbm:
    successful_records_cpbm,failed_records_cpbm=records_insert(sf,cs_obj_cpbm)
if successful_records_cpbm:    
    cpbm_record_count=len(successful_records_cpbm)
else:
    cpbm_record_count=0


In [ ]:
print(cpbm_record_count)

In [ ]:
# Try to insert failed records to ECOMM
cs_obj_ecomm=[]
failed_records=[]
for record,errors in failed_records_cpbm:
    if 'Product__r' in record:
        record['Product__r']=json.loads('{"UniqueKeyRecTypeMID__c":"012f1000000PVPiAAO' + record['MID__c'][3:]+'"}')
    cs_obj_ecomm.append(record)

    failed_records.append(errors)

if cs_obj_ecomm:
    successful_records_ecomm,failed_records_ecomm=records_insert(sf,cs_obj_ecomm)
if successful_records_ecomm:
    ecomm_record_count=len(successful_records_ecomm)
else:
    ecomm_record_count=0

if failed_records_ecomm:
    failed_record_count=len(failed_records_ecomm)
else:
    failed_record_count=0

for record,errors in failed_records_ecomm:
    failed_ids.append(record['MID__c'][3:])

t1 = ''.join(errorlist)

In [ ]:
print(ecomm_record_count)

In [ ]:
print(errorlist)
print(failed_ids)

In [ ]:
# Send email to stakeholders
if len(errorlist)==0:
    message2 = f"""customer_success_enrichment Az Function ran successfully. 
    \n Customer success objects created: {initial_record_count} 
    \n CPBM Customer success objects created: {cpbm_record_count} 
    \n eComm Customer success objects created: {ecomm_record_count}
    \n Failed records count: {failed_record_count} 
    """
    print(message2)
else:
    t1 = "".join(str(element) for element in errorlist)
    message = f'Please check customer_success_enrichment Az Function, Error is: ' + t1 
    print(t1)
    
cc_email = 'dileep.cherukuri@e-hps.com'

sub1  = "TDT customer success objects created"
body1 = """Customer success objects created on """ + pd.Timestamp("today").strftime("%m-%d-%Y") + f"""\nCustomer success objects created: {initial_record_count} \nCPBM Customer success objects created: {cpbm_record_count} \neComm Customer success objects created: {ecomm_record_count} \nFailed records count: {failed_record_count}"""

message = Mail(
    from_email='bi@heartland.us',
    to_emails=[To('diedre.hachtel@e-hps.com'),To('michael.moser@e-hps.com')],
    subject=sub1,
    plain_text_content=body1
)
if cc_email:
    message.add_cc(cc_email)

if(len(failed_ids)>0):
    failed_id_df = pd.DataFrame({'MID':failed_ids})
    base64_csv = base64.b64encode(failed_id_df.to_csv(index=False).encode()).decode()

    message.attachment = Attachment(FileContent(base64_csv),
                        FileName('Failed_MIDs.csv'),
                        FileType('text/csv'),
                        Disposition('attachment'),
                        ContentId('dataframe'))


sg = SendGridAPIClient(sendgrid_api_key)
response = sg.send(message)

In [ ]:
print("Email sent.")